In [14]:
import csv
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta
from tqdm import tqdm
import json
df = pd.read_csv('data_processing1.csv')

In [15]:
df.head(1)

,tm,ta,rn,ws,wd,hm,pa,dc10Tca,dmstMtphNo,aplYmd,lareaNm,mareaNm,wetrCd,wetrTxt,wetrSplit,new_list
0,2019-12-29 00:00,1.4,0.0,1.8,50.0,57,1018.2,3.0,NaN,20191229,서울특별시,종로구,9.0,흐림/비,"['흐림', '비']","['92', '01']"


In [16]:
# 날씨1 날씨2 컬럼만들기
df["weather_1"] = None
df["weather_2"] = None

In [17]:
for i in tqdm(range(len(df))):
    if pd.isna(df.loc[i, "dc10Tca"])==False:
        value = df.loc[i,"dc10Tca"]
        if value<6:
            df.loc[i,"weather_1"] = "맑음"
        elif value<9:
            df.loc[i,"weather_1"] = "구름많음"
        else:
            df.loc[i,"weather_1"] = "흐림"
    else:
        df.loc[i,"weather_1"] = "모름"

100%|███████████████████████████████████████████████████████████████████████████| 37776/37776 [00:06<00:00, 5861.20it/s]


In [18]:
df["풍속"] = None

In [19]:
df = df.drop(columns=["lareaNm","mareaNm"],axis=1)

In [20]:
for i in tqdm(range(len(df))):
    if pd.isna(df.loc[i, "ws"])==False:
        value = df.loc[i,"ws"]
        if value<9:
            df.loc[i,"풍속"] = "약한 강한 바"
        elif value<14:
            df.loc[i,"풍속"] = "강한 바람"
        else:
            df.loc[i,"풍속"] = "매우 강한 바람"
    else:
        df.loc[i,"풍속"] = "모름"

100%|███████████████████████████████████████████████████████████████████████████| 37776/37776 [00:05<00:00, 6508.78it/s]


In [21]:
df["풍향"] = None

In [22]:
for i in tqdm(range(len(df))):
    if pd.isna(df.loc[i, "ws"])==False:
        value = df.loc[i,"ws"]
        if value<9:
            df.loc[i,"풍속"] = "약한 강한 바람"
        elif value<14:
            df.loc[i,"풍속"] = "강한 바람"
        else:
            df.loc[i,"풍속"] = "매우 강한 바람"
    else:
        df.loc[i,"풍속"] = "모름"

100%|███████████████████████████████████████████████████████████████████████████| 37776/37776 [00:05<00:00, 6647.57it/s]


In [23]:
wd_where = [20,50,70,90,110,140,160,180,200,230,250,270,290,320,240,360]

In [24]:
wd_direction = {"20": "NNE",
"50": "NE",
"70": "ENE",
"90": "E",
"110": "ESE",
"140": "SE",
"160": "SSE",
"180": "S",
"200": "SSW",
"230": "SW",
"250": "WSW" ,
"270": "W",
"290": "WNW",
"320": "NW",
"340": "NNW",
"360": "N"}

In [25]:
for i in tqdm(range(len(df))):
    if pd.isna(df.loc[i, "wd"])==False:
        value = df.loc[i,"wd"]
        diff_list = abs(wd_where - value)
        min_ = min(diff_list)
        # print(min_)
        diff_list = diff_list.tolist()
        index = diff_list.index(min_)
        # print(wd_direction[str(wd_where[index])])
        df.loc[i,"풍향"] = wd_direction[str(wd_where[index])]
    else:
        df.loc[i,"풍향"] = "모름"

100%|███████████████████████████████████████████████████████████████████████████| 37776/37776 [00:06<00:00, 5697.97it/s]


In [26]:
df.head(1)

,tm,ta,rn,ws,wd,hm,pa,dc10Tca,dmstMtphNo,aplYmd,wetrCd,wetrTxt,wetrSplit,new_list,weather_1,weather_2,풍속,풍향
0,2019-12-29 00:00,1.4,0.0,1.8,50.0,57,1018.2,3.0,NaN,20191229,9.0,흐림/비,"['흐림', '비']","['92', '01']",맑음,None,약한 강한 바람,NE


In [27]:
for i in range(len(df)):
    if "05" in df.loc[i, "new_list"]: #그 시간대에 일별데이터에 눈이 포함되어있고
        if df.loc[i,"rn"] > 0 :
            df.loc[i,"weather_2"] = "눈"
        

In [28]:
for i in range(len(df)):
    if "01" in df.loc[i, "new_list"]: #그 시간대에 일별데이터에 비이 포함되어있고
        if df.loc[i,"rn"] > 0 :
            df.loc[i,"weather_2"] = "비"

In [29]:
df["weather_2"] = df["weather_2"].fillna("둘다아님")

In [30]:
df.isna().sum()

tm                0
ta                0
rn                0
ws               68
wd               68
hm                0
pa                8
dc10Tca           5
dmstMtphNo    27246
aplYmd            0
wetrCd            0
wetrTxt           0
wetrSplit         0
new_list          0
weather_1         0
weather_2         0
풍속                0
풍향                0
dtype: int64

In [31]:
df.to_csv('data_processing2.csv', index=False, encoding='utf-8-sig')

In [32]:
df.head()

,tm,ta,rn,ws,wd,hm,pa,dc10Tca,dmstMtphNo,aplYmd,wetrCd,wetrTxt,wetrSplit,new_list,weather_1,weather_2,풍속,풍향
0,2019-12-29 00:00,1.4,0.0,1.8,50.0,57,1018.2,3.0,NaN,20191229,9.0,흐림/비,"['흐림', '비']","['92', '01']",맑음,둘다아님,약한 강한 바람,NE
1,2019-12-29 01:00,1.2,0.0,1.8,70.0,54,1017.9,6.0,NaN,20191229,9.0,흐림/비,"['흐림', '비']","['92', '01']",구름많음,둘다아님,약한 강한 바람,ENE
2,2019-12-29 02:00,1.3,0.0,2.4,70.0,53,1017.9,4.0,NaN,20191229,9.0,흐림/비,"['흐림', '비']","['92', '01']",맑음,둘다아님,약한 강한 바람,ENE
3,2019-12-29 03:00,1.4,0.0,1.8,70.0,52,1017.9,5.0,NaN,20191229,9.0,흐림/비,"['흐림', '비']","['92', '01']",맑음,둘다아님,약한 강한 바람,ENE
4,2019-12-29 04:00,1.4,0.0,2.6,70.0,51,1017.1,5.0,NaN,20191229,9.0,흐림/비,"['흐림', '비']","['92', '01']",맑음,둘다아님,약한 강한 바람,ENE


tm 날짜
ta 기온
rn 강수량
ws 풍속
wd 풍향
 hm 습도
 pa 현지기압
 ss 일조
 dc10Tca 전운량
 dmstMtphNo 현상번호 - 눈 비인지 알수있음
 aplYmd 날짜
 lareaNm 서울특별시 고정
 mareaNm 중구 고정
 wetrCd 날씨코드
 wetrTxt 날씨코드설명

날씨1 - 구름많음 흐림 맑음 - 전우량 
날씨2 - 비, 눈<- 네이버 일별dmts
기압1000이하고 영하면=> 눈
기압 1000이하고 영상=>비
기압 1000이상이면 전운량따라 구름많음,흐름,맑음